In [9]:
import os
import pandas as pd
from tqdm.auto import tqdm
from traffic.core import Traffic, Flight
from traffic.data import opensky, navaids, airports
from typing import Union, List, Tuple
from traffic.core.mixins import PointMixin
from utils import helperfunctions as hf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
# Paras
airport = "LIRF"
folder_daily = f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/daily/"
date_range = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# # Fetching in daily packages
# for date in date_range:
#     path = f"{folder_daily}{date.month}/"
#     if not os.path.exists(path):
#         os.makedirs(path)
#     file = path + date.strftime('%Y-%m-%d')+".parquet"
#     if not os.path.exists(file):
#         print(f"fetching {date}")
#         try:
#             t = opensky.history(
#                 start=date.strftime('%Y-%m-%d 00:00'),
#                 stop=date.strftime('%Y-%m-%d 23:59'),
#                 bounds=airports[airport].shape.buffer(2).bounds,
#             )
#             t.to_parquet(file)
#         except:
#             print(f"failed to fetch {date}")

### Processing (clean invalid, assign id, only landings at airport, add landing runway)

In [11]:
# Reduction to landing 04 in monthly packages-----------------------------------
folder_list = os.listdir(folder_daily)
# for each month
# for folder in folder_list:
for folder in ['10', '11']:
    # List all files in the folder
    file_list = os.listdir(folder_daily + folder)
    filepaths = [os.path.join(folder_daily + folder, file) for file in file_list]

    # Merging to one trafic object
    t = Traffic(
        pd.concat(
            [Traffic.from_file(file).data for file in tqdm(filepaths)],
            ignore_index=True,
        )
    )

    # Cleaning
    t = t.clean_invalid().assign_id().eval(desc="Cleaning", max_workers=20)

    # Reduce to landing 04
    t = (
        t.iterate_lazy()
        .pipe(hf.has_landing_at, "LIRF")
        .eval(desc="landing at LIRF", max_workers=20)
    )

    # Save
    t.to_parquet(
        f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly/landing_{folder}.parquet"
    )

100%|██████████| 31/31 [00:53<00:00,  1.73s/it]


### Merge monthly into one yearly

In [ ]:
# Merge all monthly packages to one yearly package------------------------------
file_list = os.listdir(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly/")
file_list = [file for file in file_list if file.startswith("landing_")]
filepaths = [
    os.path.join(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly", file)
    for file in file_list
]
t = Traffic(
    pd.concat(
        [Traffic.from_file(file).data for file in tqdm(filepaths)],
        ignore_index=True,
    )
)
t.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all.parquet")